# 1 The objective of this notebook is to analyze the data from the dataset and set up a reference metric with a baseline mode

## Data Atributes/Description:

Attributes:
 * id : unique identifier for candidate (numeric)

 * job_title : job title for candidate (text)

 * location : geographical location for candidate (text)

 * connections: number of connections candidate has, 500+ means over 500 (text)

Output (desired target):
 * fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”





In [5]:
#---------Importing libraries---------#

#---Data analysis---#
import pandas as pd
import numpy as np


#---Data visualization---#
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#---NLP---#
import re
import string
from collections import Counter

import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from sentence_transformers import SentenceTransformer


#---utils---#
import os
from pathlib import Path

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\joao-
[nltk_data]     lemos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Step 1: Data Loading and Inspection

Load the dataset into a pandas DataFrame.

Check the dimensions of the dataset (number of rows and columns).

Examine the first few rows of the dataset to get an initial understanding of the data.

Check for any missing values in the dataset.

In [6]:
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw', 'potential-talents - Aspiring human resources - seeking human resources.csv')

df = pd.read_csv(raw_data_path)

df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [7]:
df.dtypes

id              int64
job_title      object
location       object
connection     object
fit           float64
dtype: object

In [8]:
#onyl object cols
df[df.columns[df.dtypes == 'object']].describe()

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [10]:
df.describe(include=object)

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [12]:
#checking for missing values
df.isnull().sum()

id              0
job_title       0
location        0
connection      0
fit           104
dtype: int64

## Step 2: Data Cleaning and Preprocessing

Handle missing values, if any, by either imputing them or removing rows/columns with missing values, depending on the situation.

Convert categorical variables into appropriate data types (e.g., convert 'month' and 'contact' to categorical data type).

Encode binary categorical variables ('default', 'housing', 'loan', and 'y') into numeric format (0 and 1) for modeling purposes.